

- cf. [Google Colab で LangGraph を試す](https://note.com/npaka/n/n053a3cb78311?sub_rt=share_h)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

assert "OPENAI_API_KEY" in os.environ
assert "TAVILY_API_KEY" in os.environ

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [3]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [5]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [6]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


workflow = StateGraph(AgentState)

In [7]:
import json
from langgraph.prebuilt import ToolInvocation
from langchain_core.messages import FunctionMessage


# agentノードの次に呼び出されるノードを決定する関数の準備
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    # function_call がない場合は終了
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # そうでない場合は続行
    else:
        return "continue"


# agentノードの処理を定義する関数の準備
def call_model(state):
    messages = state["messages"]

    # モデルの呼び出し
    response = model.invoke(messages)

    # メッセージリストに追加するメッセージを返す
    return {"messages": [response]}


# actionノードの処理を定義する関数の準備
def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    # ツールの呼び出し
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # メッセージリストに追加するメッセージを返す
    return {"messages": [function_message]}

In [8]:
from langgraph.graph import END

# グラフに2つのノードを追加
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# agentをエントリポイントとして指定
workflow.set_entry_point("agent")

# agentノードに条件付きエッジを追加
workflow.add_conditional_edges(
    "agent",  # 開始ノード
    should_continue,  # 呼び出されるノードを決定する関数
    {"continue": "action", "end": END},  # actionノードに遷移  # 終了
)

# actionノードからagentノードへのノーマルエッジを追加
workflow.add_edge("action", "agent")

# コンパイル
app = workflow.compile()

In [9]:
from langchain_core.messages import HumanMessage

# 質問応答
inputs = {"messages": [HumanMessage(content="ぼっち・ざ・ろっくのぼっちちゃんの得意な楽器は？")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='ぼっち・ざ・ろっくのぼっちちゃんの得意な楽器は？'),
  AIMessage(content='ぼっち・ざ・ろっくのぼっちちゃんの得意な楽器はギターです。')]}

In [10]:
from langchain_core.messages import HumanMessage

# 質問応答
inputs = {"messages": [HumanMessage(content="バーンブレイバーンの主人公の名前は？")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='バーンブレイバーンの主人公の名前は？'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "バーンブレイバーン 主人公 名前"\n}', 'name': 'tavily_search_results_json'}}),
  FunctionMessage(content='[{\'url\': \'https://support.apple.com/en-us/103679\', \'content\': "20:25:42 May 14, 2029 Not EV  04 04 80 28 BF 1F 28 64 D4 8F 9A D4 D8 32 94 36 6A 82 88 56 55 3F 3B 14 30 3F 90 14 7F 5D 40 EF  15:04:56 Mar 16, 2029 Not EV  00:00:00 Jan 17, 2038 2.23.140.1.1The watchOS Trust Store contains three categories of certificates: Trusted root certificates are used to establish a chain of trust that\'s used to verify other\\xa0..."}]', name='tavily_search_results_json'),
  AIMessage(content='申し訳ありませんが、バーンブレイバーンの主人公の名前に関する情報は見つかりませんでした。')]}

In [11]:
# ストリーミング
inputs = {"messages": [HumanMessage(content="バーンブレイバーンの主人公の名前は？")]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "バーンブレイバーン 主人公 名前"\n}', 'name': 'tavily_search_results_json'}})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content="[{'url': 'https://game.watch.impress.co.jp/docs/kikaku/1562690.html', 'content': '【冬アニメ2024】大張正己氏によるロボットアニメ！ 「勇気爆発バーンブレイバーン」が2024年1月11日放送開始  TVアニメ「勇気爆発バーンブレイバーン」第3話あらすじ、先行カット、予告映像が公開 2024年1月22日 【冬アニメ2024】大張正己氏によるロボットアニメ！ 「勇気爆発バーンブレイバーン」が2024年1月11日放送開始  2024年1月22日 19:23 1月11日より毎週木曜23：56からTBS系で放映されている「勇気爆発バーンブレイバーン(以下、バーンブレイバーン)」が話題を集めている。  「バーンブレイバーン」は注目するところが多いアニメではあるが、話題を集めているのは本作が「ちょっと変」だからである。王道のロボットアニメのように見せかけ、大張氏と、主役ロボ・ブレイバーンを演じる鈴村健一さんによって「王道を突き進むからこそ7 days ago — 7 days agoイサミは陸上自衛隊の最新兵器のパイロット、コテコテの軍人であり、生真面目な性格、「リアルロボットアニメの主人公」タイプのキャラクターだ。その彼が\\xa0...'}]", name='tavily_search_results_json')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='バーンブレイバーンの主人公の名前はイサミです。')]}

---

Output from node '